# SI tenemos tiempo, usamos este metodo para sacar los features de la imagen

### Extracción y matching de features con LoFTR (estado del arte)
A continuación, se utiliza LoFTR (a través de la librería kornia) para detectar y matchear features entre las imágenes rectificadas. Es necesario instalar kornia y torch si no están instalados.

In [ ]:
# Instalar kornia y torch si es necesario (solo la primera vez)
# !pip install kornia torch --quiet

import torch
import kornia as K
import kornia.feature as KF

def loftr_match(img1, img2):
    # Convertir a tensores y normalizar
    t1 = K.image_to_tensor(img1, False).float() / 255.0  # (1,1,H,W)
    t2 = K.image_to_tensor(img2, False).float() / 255.0
    if torch.cuda.is_available():
        t1 = t1.cuda()
        t2 = t2.cuda()
    matcher = KF.LoFTR(pretrained="outdoor")
    if torch.cuda.is_available():
        matcher = matcher.cuda()
    input_dict = {"image0": t1, "image1": t2}
    with torch.no_grad():
        correspondences = matcher(input_dict)
    mkpts0 = correspondences["keypoints0"].cpu().numpy()
    mkpts1 = correspondences["keypoints1"].cpu().numpy()
    return mkpts0, mkpts1

# Matching con LoFTR para el primer par de imágenes
loftr_pts1, loftr_pts2 = loftr_match(left_img, right_img)
print(f"LoFTR encontró {loftr_pts1.shape[0]} matches.")

# Visualización de matches
import cv2
import matplotlib.pyplot as plt

def draw_loftr_matches(img1, img2, pts1, pts2, n=100):
    img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    h1, w1 = img1.shape
    h2, w2 = img2.shape
    vis = np.hstack([img1_color, img2_color])
    n = min(n, pts1.shape[0])
    idx = np.random.choice(pts1.shape[0], n, replace=False)
    for i in idx:
        pt1 = tuple(np.round(pts1[i]).astype(int))
        pt2 = tuple(np.round(pts2[i]).astype(int) + np.array([w1, 0]))
        color = tuple(np.random.randint(0, 255, 3).tolist())
        cv2.circle(vis, pt1, 3, color, -1)
        cv2.circle(vis, pt2, 3, color, -1)
        cv2.line(vis, pt1, pt2, color, 1)
    plt.figure(figsize=(15, 8))
    plt.imshow(vis[..., ::-1])
    plt.axis('off')
    plt.title('LoFTR matches (frame 1)')
    plt.show()

draw_loftr_matches(left_img, right_img, loftr_pts1, loftr_pts2, n=100)

### Extracción y matching de features con LoFTR (frame 2)
Repetimos el proceso de matching con LoFTR para el segundo par de imágenes rectificadas.

In [ ]:
# Matching con LoFTR para el segundo par de imágenes
loftr_pts1_2, loftr_pts2_2 = loftr_match(left_img_2, right_img_2)
print(f"LoFTR encontró {loftr_pts1_2.shape[0]} matches en el frame 2.")

draw_loftr_matches(left_img_2, right_img_2, loftr_pts1_2, loftr_pts2_2, n=100)

### Asociación de puntos entre frames usando LoFTR (deep learning)
Usamos LoFTR para asociar directamente la imagen izquierda del frame 1 con la del frame 2

In [ ]:
# Usamos LoFTR para asociar directamente la imagen izquierda del frame 1 con la del frame 2
loftr_pts1_temporal, loftr_pts2_temporal = loftr_match(left_img, left_img_2)
print(f"LoFTR encontró {loftr_pts1_temporal.shape[0]} matches entre frame 1 y frame 2.")

draw_loftr_matches(left_img, left_img_2, loftr_pts1_temporal, loftr_pts2_temporal, n=100)